In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import nltk
from nltk.corpus import stopwords
import string
import warnings
warnings.filterwarnings('ignore')
import re
import plotly.express as px
import wordcloud
from wordcloud import WordCloud
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
import pickle
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
import xgboost
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import mean_squared_error
import warnings, copy
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode

In [7]:
heart = pd.read_csv("C:/Users/esram/AppData/Roaming/Microsoft/Windows/Start Menu/Programs/Anaconda3 (64-bit)/heart.csv")

In [8]:
heart.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [11]:
plot_df=heart.copy()
p1=plot_df.HeartDisease.value_counts()
p1=p1.rename('count').reset_index().sort_values('count', ascending=False)
x1=p1['index'].apply(lambda x: 'Heart Disease' if x==1 else 'No Disease')

# Plot 2
p2=plot_df.groupby('HeartDisease')['Age'].median().round(0).astype(int)
p2=p2.rename('age').reset_index().sort_values('age', ascending=False)

# Plot 3
p3=plot_df.groupby('Sex')['HeartDisease'].value_counts(normalize=True)
p3=p3.mul(100).rename('pct').reset_index()
x2=p3.Sex.apply(lambda x: 'Women' if x=='F' else 'Men').unique()[::-1]
y0=p3[p3.HeartDisease==0]['pct'][::-1]
y1=p3[p3.HeartDisease==1]['pct'][::-1]


temp = dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12)))
fig = make_subplots(rows=2, cols=2,
                    vertical_spacing=0.12,
                    specs=[[{}, {}], [{'colspan': 2}, None]],
                    subplot_titles=("Number of Patients", "Average Age"))
# Plot 1
fig.add_trace(go.Bar(x=x1, y=p1['count'], text=p1['count'],texttemplate='n = %{text}', textposition='outside',
                     marker=dict(color=['#3d799d', '#c1dbe0'], opacity=0.8),
                     #hovertemplate='Number of Patients with %{x} = %{y}<extra></extra>',
                     name="Heart Disease", showlegend=False),row=1,col=1)
# Plot 2
fig.add_trace(go.Bar(x=x1, y=p2['age'], text=p2['age'], textposition='outside',
                     marker=dict(color=['#3d799d', '#c1dbe0'], opacity=0.8),
                     #hovertemplate='Median Age of Patients with %{x} = %{y} years<extra></extra>',
                     name="Heart Disease", showlegend=False),row=1,col=2)


# Plot 3
fig.add_trace(go.Bar(x=x2, y=y1, name='Heart Disease', text=y1, textposition='outside',
                     texttemplate='%{text:.1f}%', width=0.38, 
                     #hovertemplate='Proportion of %{x} with Heart Disease = %{y:.2f}%<extra></extra>',
                     marker=dict(color='#3d799d', opacity=0.8)), row=2,col=1)
fig.add_trace(go.Bar(x=x2, y=y0, name='No Disease', text=y0, textposition='outside',
                     texttemplate='%{text:.1f}%', width=0.38, 
                     #hovertemplate='Proportion of %{x} without Heart Disease = %{y:.2f}%<extra></extra>',
                     marker=dict(color='#c1dbe0', opacity=0.8)), row=2,col=1)


fig.update_traces(marker=dict(line=dict(width=1, color='#000000')))
fig.update_layout(title="Heart Disease Statistics", showlegend=True, template=temp, 
                  legend=dict(orientation="h", yanchor="bottom", y=.4, xanchor="right", x=.97),
                  barmode='group', bargap=.15, height=1000, width=700)
fig.update_yaxes(title="", tickmode = 'array', range=(0, 570), dtick=5, row=1,col=1)
fig.update_yaxes(title="Age, in years", tickmode = 'array', range=(0, 65), dtick=6, row=1,col=2)
fig.update_yaxes(title="", ticksuffix='%',tickmode = 'array', range=(0, 89), dtick=5, row=2,col=1)
fig.show()

In [12]:
# Plot 1
p1=heart.groupby('HeartDisease')['ChestPainType'].value_counts(normalize=True)
p1=p1.mul(100).rename('pct').reset_index()
x1=p1.HeartDisease.apply(lambda x: 'Heart Disease' if x==1 else 'No Disease').unique()[::-1]
y0=p1[p1.ChestPainType=='ASY']['pct'][::-1]
y1=p1[p1.ChestPainType=='NAP']['pct'][::-1]
y2=p1[p1.ChestPainType=='ATA']['pct'][::-1]
y3=p1[p1.ChestPainType=='TA']['pct'][::-1]

# Plot 2
p2=heart.groupby('HeartDisease')['Sex'].value_counts(normalize=True)
p2=p2.mul(100).rename('pct').reset_index()
x2=p2.HeartDisease.apply(lambda x: 'Heart Disease' if x==1 else 'No Disease').unique()[::-1]
y4=p2[p2.Sex=='M']['pct'][::-1]
y5=p2[p2.Sex=='F']['pct'][::-1]


fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=("Prevalence of Chest Pain in Heart Disease", 
                                    "Gender Distribution in Heart Disease"))
# Plot 1
fig.append_trace(go.Bar(x=x1, y=y0, name='Asymptomatic', text=y0, textposition='outside',
                     texttemplate='%{text:.3s}%', width=0.2, legendgroup = '1',
                     hovertemplate='Prevalence of Asymptomatic Chest Pain in Patients with %{x} = %{y:.2f}%',
                     marker=dict(color='#3d799d', opacity=0.8)), row=1,col=1) #E2C3B5
fig.append_trace(go.Bar(x=x1, y=y1, name='Non-Anginal Pain', text=y1, textposition='outside',
                     texttemplate='%{text:.3s}%', width=0.2, legendgroup = '1',
                     hovertemplate='Prevalence of Non-Anginal Pain in Patients with %{x} = %{y:.2f}%',
                     marker=dict(color='#5882b0', opacity=0.8)), row=1,col=1)
fig.append_trace(go.Bar(x=x1, y=y2, name='Atypical Angina', text=y2, textposition='outside',
                     texttemplate='%{text:.3s}%', width=0.2, legendgroup = '1',
                     hovertemplate='Prevalence of Atypical Angina in Patients with %{x} = %{y:.2f}%',
                     marker=dict(color='#b5e3d8', opacity=0.8)), row=1,col=1)
fig.append_trace(go.Bar(x=x1, y=y3, name='Typical Angina', text=y3, textposition='outside',
                     texttemplate='%{text:.3s}%', width=0.2, legendgroup = '1',
                     hovertemplate='Prevalence of Typical Angina in Patients with %{x} = %{y:.2f}%',
                     marker=dict(color='#c1dbe0', opacity=0.8)), row=1,col=1)
# Plot 2
fig.append_trace(go.Bar(x=x2, y=y4, name='Men', text=y4, textposition='outside',
                     texttemplate='%{text:.3s}%', width=0.4, legendgroup = '2',
                     hovertemplate='Proportion of %{x} among Men = %{y:.2f}%',
                     marker=dict(color='#3d799d', opacity=0.8)), row=2,col=1)
fig.append_trace(go.Bar(x=x2, y=y5, name='Women', text=y5, textposition='outside',
                     texttemplate='%{text:.3s}%', width=0.4, legendgroup = '2',
                     hovertemplate='Proportion of %{x} among Women = %{y:.2f}%',
                     marker=dict(color='#c1dbe0', opacity=0.8)), row=2,col=1)

fig.update_traces(marker=dict(line=dict(width=1, color='#000000')))
fig.update_layout(showlegend=True, template=temp, legend_tracegroupgap = 450, legend_title="Chest Pain Type",
                  barmode='group', bargap=.15, height=1000, width=700,
                  yaxis1_range=[0,90], yaxis1_ticksuffix='%',
                  yaxis2_range=[0,99], yaxis2_ticksuffix='%')
fig.show()

In [13]:
# Correlations
corr=heart.corr()
x = corr.columns.tolist() 
y = corr.index.tolist()
z = corr.values
text = corr.values.round(2)

fig = ff.create_annotated_heatmap(z=z, x=x, y=y, annotation_text=text, colorscale='matter', 
                                  reversescale=True, showscale=True,
                                  hovertemplate="Correlation of %{x} and %{y}= %{z:.3f}")
fig.update_layout(template=temp, title="Heart Disease Correlations")
fig.show()